## Prerequisites

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.
- gemini only works in certain regions, so you may need to use vpn to access it.

Import the necessary packages.

In [15]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
from dotenv import load_dotenv, find_dotenv
import os

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Setup your API key


In [16]:
load_dotenv(find_dotenv())

True

In [19]:
GOOGLE_API_KEY = os.environ.get('GEMINI_API_KEY')

In [21]:
genai.configure(api_key=GOOGLE_API_KEY)

## List models


* `gemini-pro`: optimized for text-only prompts.
* `gemini-pro-vision`: optimized for text-and-images prompts.

In [26]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-pro
models/gemini-pro-vision


In [27]:
model = genai.GenerativeModel('gemini-pro')

In [32]:
lst = ["bonjour luxair five five one identified cleared passeiry torino flight level three three zero", 
"lufthansa four seven two three resume own navigation to willisau"]

In [40]:
atc_instructions = """
Entering and leaving airspace ATC instruction to a pilot from uncontrolled zone:

  * ENTER CONTROLLED AIRSPACE (or CONTROL ZONE) [VIA (significant point or route)] AT (level) [AT (time)]
  * LEAVE CONTROLLED AIRSPACE (or CONTROL ZONE) [VIA (significant point or route)] AT (level) (or CLIMBING, or DESCENDING)

ATC instruction to instruct a pilot to join a specific point with restrictions given by ATC:
"""

# Now atc_instructions is a string containing your provided text

In [45]:
tenerife_txt = """Communication without LLM:
 
Ambiguity in Clearance Request and Response:
KLM (Captain): "We are now at takeoff."
ATC: "OK... standby for takeoff, I will call you."
Error Type: Ambiguity / Misinterpretation
What Went Wrong: The KLM captain's statement was ambiguous, potentially implying they were initiating takeoff. The tower's response, "OK," might have been misinterpreted by the KLM crew as a takeoff clearance, though the controller intended them to wait.
Misinterpretation of Clearance:
ATC: "Papa Alpha 1736 report runway clear."
Pan Am: "OK, will report when we're clear."
Error Type: Misinterpretation / Lack of Explicit Confirmation
What Went Wrong: There was no explicit confirmation from Pan Am that they were still on the runway, and KLM did not hear their transmission indicating they were still taxiing.
Non-Standard Phraseology and Assumption:
KLM (Captain): "We are at takeoff."
Error Type: Non-Standard Phraseology / Assumption
What Went Wrong: The captain's use of "We are at takeoff" was non-standard and ambiguous. It was not a standard request or confirmation of clearance for takeoff.
Incomplete Readback and Missed Correction:
ATC to KLM: "KLM 4805, you are cleared to the Papa beacon. Climb to and maintain flight level 90, right turn after takeoff, proceed with heading 040 until intercepting the 325 radial from Las Palmas VOR."
KLM (Co-pilot): "Ah, roger, sir, we are cleared to the Papa beacon flight level 90, right turn out 040 until intercepting the 325."
Error Type: Incomplete Readback / Lack of Correction
What Went Wrong: The co-pilot's readback was not complete and did not include a takeoff clearance, but the controller did not correct or clarify this.
Critical Misunderstanding in the Final Moments:
KLM (Captain): "Let's go, check thrust."
Error Type: Critical Misunderstanding / Assumption
What Went Wrong: The KLM captain assumed clearance for takeoff without explicit confirmation from the tower or assurance that the runway was clear. This assumption led directly to the collision.


Communication with LLM
 
Initial Ambiguous Communication
KLM (Captain): "We are now at takeoff."
LLM (in KLM Cockpit): "[Ambiguity Detected] Captain, your statement is ambiguous and might be misunderstood as a takeoff initiation. Please clarify your intention. You should say, 'Requesting takeoff clearance' to explicitly seek permission."
KLM (Captain, corrected): "KLM 4805 requesting takeoff clearance."
Response from the Tower with Potential Misunderstanding
ATC: "OK... standby for takeoff, I will call you."
LLM (in KLM Cockpit): "[Potential Misunderstanding Detected] Note that the controller has instructed to 'standby for takeoff.' This is not a clearance. Remain in position until a clear takeoff clearance is given."
KLM (Captain, acknowledging): "Standing by for your takeoff call, KLM 4805."
Pan Am Reporting Position with Overlapping Transmission
Pan Am: "We're still taxiing down the runway, the Clipper 1736."
LLM (in ATC Tower): "[Critical Information] Attention tower, Pan Am 1736 is still on the runway. Ensure this is clearly communicated to KLM 4805."
ATC (acknowledging): "KLM 4805, hold position, Pan Am 1736 still on the runway."
Clearance Confirmation and Readback
ATC to KLM: "KLM 4805, you are cleared to the Papa beacon. Climb to and maintain flight level 90, right turn after takeoff, proceed with heading 040 until intercepting the 325 radial from Las Palmas VOR."
KLM (Co-pilot): "Ah, roger, sir, we are cleared to the Papa beacon flight level 90, right turn out 040 until intercepting the 325."
LLM (in KLM Cockpit): "[Incomplete Readback Detected] Co-pilot, your readback missed the takeoff clearance. Please confirm 'Cleared for takeoff' with the tower."
KLM (Co-pilot, corrected): "And confirming, we are cleared for takeoff, KLM 4805?"
Final Confirmation Before Takeoff
KLM (Captain): "Let's go, check thrust."
LLM (in KLM Cockpit): "[Final Confirmation Needed] Captain, final confirmation from the tower for takeoff clearance is required. The runway status should be reconfirmed."
KLM (Captain, to Tower): "KLM 4805 requesting final confirmation for takeoff, is the runway clear?"
"""

In [41]:
response = model.generate_content(f'based on theis manual: {atc_instructions} generate me a correct command, incorrect command and a reason explaining why the incorrect command is incorrect')

In [43]:
to_markdown(response.text)

> **Correct Command**:
> 
> "ENTER CONTROLLED AIRSPACE VIA BRAZOT POINT AT FIVE THOUSAND FEET AT ONE-SIX-ZERO-ZERO HOURS."
> 
> **Explanation**:
> 
> * The command clearly specifies the airspace to be entered (controlled airspace).
> * It provides a specific entry point (BRAZOT POINT).
> * It assigns a specific altitude (FIVE THOUSAND FEET).
> * It includes a specific time (ONE-SIX-ZERO-ZERO HOURS) for the entry.
> 
> **Incorrect Command**:
> 
> "ENTER CONTROLLED AIRSPACE AT FOUR THOUSAND FEET."
> 
> **Explanation**:
> 
> * The command lacks a specific entry point.
> * It does not include a time for the entry.
> * It provides an altitude that is not consistent with the aircraft's current altitude, potentially leading to conflict with other aircraft.
> 
> Incorrect commands can result in confusion, misunderstandings, and potential safety hazards. Therefore, it is essential for ATC instructions to be clear, concise, and complete, providing all necessary information for the pilot to safely and efficiently enter or leave controlled airspace.

In [51]:
response2 = model.generate_content(f'based on this example of how LLM should work: {tenerife_txt} generate me a correct ATC command, incorrect ATC command and a reason explaining why the incorrect command is incorrect so I could have a dataset for fine tuning models. Provide me in a table format. Provide 10 examples')
to_markdown(response2.text)

> | Correct ATC Command | Incorrect ATC Command | Reason |
> |---|---|---|
> | "KLM 4805, cleared for takeoff, runway 25L." | "KLM 4805, takeoff, runway 25L." | The incorrect command is missing the word "cleared," which is essential for giving permission to take off. |
> | "Cessna 172, taxi to runway 18 via taxiway Alpha." | "Cessna 172, taxi runway 18 via taxiway Alpha." | The incorrect command is missing the word "to," which is necessary to indicate the destination of the taxi instruction. |
> | "Airbus A320, hold short of runway 09." | "Airbus A320, hold runway 09." | The incorrect command is missing the word "short," which is crucial for instructing the aircraft to stop before entering the runway. |
> | "Boeing 737, descend and maintain 10,000 feet." | "Boeing 737, descend 10,000 feet." | The incorrect command is missing the word "maintain," which is necessary to indicate that the aircraft should level off at 10,000 feet. |
> | "Embraer 190, turn left heading 270." | "Embraer 190, left 270." | The incorrect command is missing the word "turn," which is essential for instructing the aircraft to change direction. |
> | "Gulfstream G550, contact approach control on frequency 121.5 MHz." | "Gulfstream G550, approach 121.5." | The incorrect command is missing the words "contact" and "control," which are necessary to indicate that the aircraft should establish communication with approach control. |
> | "Piper Archer, climb and maintain 5,500 feet." | "Piper Archer, climb 5,500." | The incorrect command is missing the word "maintain," which is necessary to indicate that the aircraft should level off at 5,500 feet. |
> | "Daher TBM 930, expedite your climb to flight level 250." | "Daher TBM 930, expedite climb 250." | The incorrect command is missing the word "to," which is necessary to indicate the target altitude for the climb. |
> | "ATR 72, reduce speed to 180 knots." | "ATR 72, speed 180." | The incorrect command is missing the word "reduce," which is necessary to indicate that the aircraft should slow down. |
> | "Boeing 777, maintain heading 060 until reaching the VOR." | "Boeing 777, maintain 060 VOR." | The incorrect command is missing the word "heading," which is necessary to indicate the direction that the aircraft should maintain. |

In [47]:
to_markdown(response.text)

> **Correct Command**:
> 
> "ENTER CONTROLLED AIRSPACE VIA BRAZOT POINT AT FIVE THOUSAND FEET AT ONE-SIX-ZERO-ZERO HOURS."
> 
> **Explanation**:
> 
> * The command clearly specifies the airspace to be entered (controlled airspace).
> * It provides a specific entry point (BRAZOT POINT).
> * It assigns a specific altitude (FIVE THOUSAND FEET).
> * It includes a specific time (ONE-SIX-ZERO-ZERO HOURS) for the entry.
> 
> **Incorrect Command**:
> 
> "ENTER CONTROLLED AIRSPACE AT FOUR THOUSAND FEET."
> 
> **Explanation**:
> 
> * The command lacks a specific entry point.
> * It does not include a time for the entry.
> * It provides an altitude that is not consistent with the aircraft's current altitude, potentially leading to conflict with other aircraft.
> 
> Incorrect commands can result in confusion, misunderstandings, and potential safety hazards. Therefore, it is essential for ATC instructions to be clear, concise, and complete, providing all necessary information for the pilot to safely and efficiently enter or leave controlled airspace.

In [39]:
%%time
response_lst = []
for i in lst:
    response = model.generate_content(f"{i}\n. Decide if this command is correct or not. If it is correct you need to modify in order to make it incorrect command and vice versa. The format should be in a table with columns: correct command, incorrect command and why incorrect. Only focus now on terminology and phraseology")
    print(response.text)
    response_lst.append(response)

| Correct Command | Incorrect Command | Why Incorrect |
|---|---|---|
| bonjour luxair five five one identified cleared passeiry torino flight level three three zero | cleared luxair five five one identified passeiry torino flight level three three zero | The word "cleared" should not be used before the callsign. |
| bonjour luxair five five one identified cleared passeiry torino flight level three three zero | bonjour luxair five five one passeiry torino flight level three three zero | The word "identified" should not be used after the callsign. |
| bonjour luxair five five one identified cleared passeiry torino flight level three three zero | bonjour luxair five five one passeiry torino | The word "flight level" should be used before the altitude. |
| bonjour luxair five five one identified cleared passeiry torino flight level three three zero | bonjour luxair five five one identified cleared passeiry torino three three zero | The word "flight level" should be used before the altitud

In [37]:
response = model.generate_content("what is 2+2")

In [38]:
to_markdown(response.text)

> 4

In [36]:
for response in response_lst:
    to_markdown(response.text)